In [1]:
import torch
import torch.nn as nn

from torch_frft.frft_module import frft
from torch_frft.dfrft_module import dfrft
from torch_frft.layer import DFrFTLayer, FrFTLayer

In [2]:
num_samples = 100
seq_length = 16
a_original = torch.tensor(1.45, dtype=torch.float32)
X = torch.randn(num_samples, seq_length, dtype=torch.float32)
Y = frft(X, a_original)

model = nn.Sequential(FrFTLayer(order=1.25))
optim = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 1000

for epoch in range(1 + epochs):
    optim.zero_grad()
    loss = torch.norm(Y - model(X))
    loss.backward()
    optim.step()

    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")

print("Original  a:", a_original)
print("Estimated a:", model[0].order)

Epoch [0/1000], Loss: 59.5393
Epoch [100/1000], Loss: 30.9991
Epoch [200/1000], Loss: 0.1275
Epoch [300/1000], Loss: 0.0764
Epoch [400/1000], Loss: 0.0969
Epoch [500/1000], Loss: 0.0780
Epoch [600/1000], Loss: 0.0846
Epoch [700/1000], Loss: 0.0837
Epoch [800/1000], Loss: 0.0748
Epoch [900/1000], Loss: 0.0898
Epoch [1000/1000], Loss: 0.0663
Original  a: tensor(1.4500)
Estimated a: Parameter containing:
tensor(1.4498, requires_grad=True)


In [3]:
num_samples = 100
seq_length = 16
a_original = torch.tensor(1.1, dtype=torch.float32)
X = torch.randn(num_samples, seq_length, dtype=torch.float32)
Y = dfrft(X, a_original, dim=-1)

model = nn.Sequential(DFrFTLayer(order=1.25, dim=-1))
optim = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 1000

for epoch in range(1 + epochs):
    optim.zero_grad()
    loss = torch.norm(Y - model(X))
    loss.backward()
    optim.step()

    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")

print("Original  a:", a_original)
print("Estimated a:", model[0].order)

Epoch [0/1000], Loss: 60.2955
Epoch [100/1000], Loss: 14.5485
Epoch [200/1000], Loss: 0.2127
Epoch [300/1000], Loss: 0.1997
Epoch [400/1000], Loss: 0.1942
Epoch [500/1000], Loss: 0.1913
Epoch [600/1000], Loss: 0.1895
Epoch [700/1000], Loss: 0.1882
Epoch [800/1000], Loss: 0.1872
Epoch [900/1000], Loss: 0.1866
Epoch [1000/1000], Loss: 0.1861
Original  a: tensor(1.1000)
Estimated a: Parameter containing:
tensor(1.1003, requires_grad=True)
